In [1]:
!pip install category_encoders==2.*
#!pip install pandas-profiling==2.*

In [2]:
import requests
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
#import os
#from dotenv import load_dotenv
#load_dotenv()
#API_KEY = os.getenv("API_KEY")

API_KEY = 'uEH5NtA'

request_url = f"http://strainapi.evanbusse.com/{API_KEY}/strains/search/all"
response = requests.get(request_url)
data = json.loads(response.text)

print(request_url)
print(type(response)) #> <class 'requests.models.Response'>
print(response.status_code) #> 200
print(type(response.text)) #> <class 'str'>
print(type(data)) #> <class 'dict'>

http://strainapi.evanbusse.com/uEH5NtA/strains/search/all
<class 'requests.models.Response'>
200
<class 'str'>
<class 'dict'>


In [17]:
# All strains
strain_names = pd.DataFrame(list(data.keys()), columns =['name'])
strain_names.head()

,name
0,Afpak
1,African
2,Afternoon Delight
3,Afwreck
4,Agent Orange


In [18]:
# Get strain characteristics
strain_values = list(data.values())
strain_values = pd.DataFrame.from_dict(strain_values)
strain_values.head()

,id,race,flavors,effects
0,1,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S..."
1,2,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',..."
2,3,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',..."
3,4,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ..."
4,5,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ..."


In [56]:
# Add strain name to the dataframe
strains = pd.concat([strain_names, strain_values], axis=1)
strains.head()

,name,id,race,flavors,effects
0,Afpak,1,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S..."
1,African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',..."
2,Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',..."
3,Afwreck,4,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ..."
4,Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ..."


In [57]:
# Get a line for each flavor
strains = strains.flavors.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["flavors"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'effects'], value_name = "flavor") \
    .drop("variable", axis = 1) \
    .dropna()
strains

,name,id,race,effects,flavor
0,Afpak,1,hybrid,"{'positive': ['Relaxed', 'Hungry', 'Happy', 'S...",Earthy
1,African,2,sativa,"{'positive': ['Euphoric', 'Happy', 'Creative',...",Spicy/Herbal
2,Afternoon Delight,3,hybrid,"{'positive': ['Relaxed', 'Hungry', 'Euphoric',...",Pepper
3,Afwreck,4,hybrid,"{'positive': ['Relaxed', 'Happy', 'Creative', ...",Pine
4,Agent Orange,5,hybrid,"{'positive': ['Relaxed', 'Euphoric', 'Happy', ...",Citrus
...,...,...,...,...,...
5905,Zeus OG,2158,hybrid,"{'positive': ['Relaxed', 'Happy', 'Energetic',...",Pine
5906,Zkittlez,2159,indica,"{'positive': ['Relaxed', 'Euphoric', 'Happy', ...",Grape
5907,Zombie OG,2160,indica,"{'positive': ['Relaxed', 'Euphoric', 'Happy', ...",Sweet
5908,Zoom Pie,2161,indica,"{'positive': ['Relaxed', 'Hungry', 'Happy', 'U...",Pungent


In [58]:
# Get a line for each type of effect
strains = strains.effects.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["effects"], axis = 1)
strains

,positive,negative,medical,name,id,race,flavor
0,"[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",Afpak,1,hybrid,Earthy
1,"[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",African,2,sativa,Spicy/Herbal
2,"[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",Afternoon Delight,3,hybrid,Pepper
3,"[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",Afwreck,4,hybrid,Pine
4,"[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",Agent Orange,5,hybrid,Citrus
...,...,...,...,...,...,...,...
5905,"[Relaxed, Happy, Energetic, Uplifted, Focused]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Fatigue]",Zeus OG,2158,hybrid,Pine
5906,"[Relaxed, Euphoric, Happy, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Nausea, H...",Zkittlez,2159,indica,Grape
5907,"[Relaxed, Euphoric, Happy, Sleepy, Giggly]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Lack of A...",Zombie OG,2160,indica,Sweet
5908,"[Relaxed, Hungry, Happy, Uplifted, Giggly]",[],"[Insomnia, Pain, Cramps, Lack of Appetite, Mus...",Zoom Pie,2161,indica,Pungent


In [59]:
# Get a line for each positive effect
strains = strains.positive.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["positive"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'flavor', 'negative', 'medical'], value_name = "positive_effect") \
    .drop("variable", axis = 1) \
    .dropna()
strains

,name,id,race,flavor,negative,medical,positive_effect
0,Afpak,1,hybrid,Earthy,[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",Relaxed
1,African,2,sativa,Spicy/Herbal,[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",Euphoric
2,Afternoon Delight,3,hybrid,Pepper,"[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",Relaxed
3,Afwreck,4,hybrid,Pine,"[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",Relaxed
4,Agent Orange,5,hybrid,Citrus,"[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",Relaxed
...,...,...,...,...,...,...,...
28975,Zeus OG,2158,hybrid,Pine,"[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Fatigue]",Focused
28976,Zkittlez,2159,indica,Grape,"[Dizzy, Dry Mouth, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Nausea, H...",Sleepy
28977,Zombie OG,2160,indica,Sweet,"[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Insomnia, Pain, Stress, Lack of A...",Giggly
28978,Zoom Pie,2161,indica,Pungent,[],"[Insomnia, Pain, Cramps, Lack of Appetite, Mus...",Giggly


In [60]:
# Get a line for each negative effect
strains = strains.negative.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["negative"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'flavor', 'positive_effect', 'medical'], value_name = "negative_effect") \
    .drop("variable", axis = 1) \
    .dropna()
strains

,name,id,race,flavor,positive_effect,medical,negative_effect
0,Afpak,1,hybrid,Earthy,Relaxed,"[Depression, Insomnia, Pain, Stress, Lack of A...",Dizzy
1,African,2,sativa,Spicy/Herbal,Euphoric,"[Depression, Pain, Stress, Lack of Appetite, N...",Dry Mouth
2,Afternoon Delight,3,hybrid,Pepper,Relaxed,"[Depression, Insomnia, Pain, Stress, Cramps, H...",Dizzy
3,Afwreck,4,hybrid,Pine,Relaxed,"[Pain, Stress, Headache, Fatigue, Headaches, M...",Dizzy
4,Agent Orange,5,hybrid,Citrus,Relaxed,"[Depression, Pain, Stress, Nausea, Headache, H...",Dizzy
...,...,...,...,...,...,...,...
136960,White Fire OG,2097,hybrid,Sweet,Uplifted,"[Depression, Pain, Stress, Nausea, Fatigue]",Anxious
136965,White Kush,2103,indica,Sweet,Sleepy,"[Depression, Insomnia, Pain, Stress, Headaches]",Anxious
136972,White Rhino,2110,indica,Woody,Sleepy,"[Depression, Insomnia, Pain, Stress, Lack of A...",Anxious
136980,White Widow,2123,hybrid,Pungent,Uplifted,"[Depression, Pain, Stress, Lack of Appetite, F...",Anxious


In [61]:
# Get a line for each medical effect
strains = strains.medical.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["medical"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'flavor', 'positive_effect', 'negative_effect'], value_name = "illness") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values(['id', 'flavor', 'positive_effect', 'negative_effect', 'illness'], \
                 ascending = (1,1,1,1,1))
strains

,name,id,race,flavor,positive_effect,negative_effect,illness
10912,Afpak,1,hybrid,Chemical,Happy,Dizzy,Depression
77176,Afpak,1,hybrid,Chemical,Happy,Dizzy,Insomnia
275968,Afpak,1,hybrid,Chemical,Happy,Dizzy,Lack of Appetite
143440,Afpak,1,hybrid,Chemical,Happy,Dizzy,Pain
209704,Afpak,1,hybrid,Chemical,Happy,Dizzy,Stress
...,...,...,...,...,...,...,...
160694,ʻAlenuihāhā,2162,hybrid,Tropical,Happy,Dry Mouth,Fatigue
94430,ʻAlenuihāhā,2162,hybrid,Tropical,Happy,Dry Mouth,Stress
44217,ʻAlenuihāhā,2162,hybrid,Tropical,Happy,Paranoid,Depression
176745,ʻAlenuihāhā,2162,hybrid,Tropical,Happy,Paranoid,Fatigue


In [76]:
# Drop the strain names, so they won't be included in Encoder
X = strains.drop('name', axis=1)

In [77]:
encoder = OneHotEncoder() 

In [78]:
X_encoded = encoder.fit_transform(X)

In [79]:
X_dict = X.to_dict(orient='records') # turn each row as key-value pairs

In [80]:
# DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# instantiate a Dictvectorizer object for X
X_dv = DictVectorizer(sparse=False)

# sparse = False makes the output is not a sparse matrix

In [81]:
# apply strains_dv on strains_dict
X_encoded = X_dv.fit_transform(X_dict)

# vocabulary
vocab = X_dv.vocabulary_

# Get dummies
X = pd.get_dummies(X, prefix_sep='_')
# X head
X.head()

,id,race_hybrid,race_indica,race_sativa,flavor_Ammonia,flavor_Apple,flavor_Apricot,flavor_Berry,flavor_Blue Cheese,flavor_Blueberry,...,illness_Headaches,illness_Inflammation,illness_Insomnia,illness_Lack of Appetite,illness_Muscle Spasms,illness_Nausea,illness_Pain,illness_Seizures,illness_Spasticity,illness_Stress
10912,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77176,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
275968,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
143440,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
209704,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [82]:
print(strains.shape)
print(X.shape)

(353571, 7)
(353571, 85)
